Data sourced from https://www.kaggle.com/datasets/yash612/stockmarket-sentiment-dataset?resource=download. Data processing stolen, abridged from https://www.kaggle.com/code/juniorbueno/stock-market-sentimen-bert-tokenizer

You may need to install (via pip3):
torch
matplotlib
numpy
nltk

In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import csv
import re
from nltk import stem
from nltk.corpus import stopwords
from collections import Counter
from copy import deepcopy

stemm = stem.porter.PorterStemmer()
stops = set(stopwords.words('english'))

with open('stock_data.csv', encoding='utf8') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    data = list(readCSV)

data = list(zip(*data))
text = data[0][1:]
sentiment = data[1][1:]

In [44]:
# Hyperparameters
dict_word_frequency_requirement = 5

In [45]:
def clean_tweet(tweet):
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[']", "", tweet)
  tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet)
  tweet = re.sub(r"([^a-zA-Z])", r" \1 ", tweet)
  tweet = re.sub(r" +", ' ', tweet)
  tweet = tweet.lower()
  return tweet

In [46]:
def tokenize(tweet):
    tweet = clean_tweet(tweet)
    tweet = filter(lambda w: w not in stops, tweet.strip().split())
    return list(map(stemm.stem, tweet))

In [47]:
test = text[1]
print(test)
print(tokenize(test))

user: AAP MOVIE. 55% return for the FEA/GEED indicator just 15 trades for the year.  AWESOME.  
['user', 'aap', 'movi', '.', 'return', 'fea', 'geed', 'indic', 'trade', 'year', '.', 'awesom', '.']


In [48]:
tokenized_tweets = map(tokenize, text)

In [49]:
word_count = Counter()
for data in deepcopy(tokenized_tweets):
    word_count.update(data)

vocab = [word for word in word_count if word_count[word] >= dict_word_frequency_requirement]
dictionary = {word : i+1 for i, word in enumerate(vocab)}
len(dictionary)

1690

In [50]:
encoded_tweets = [list(map(lambda w : dictionary.get(w, 0), tweet)) for tweet in tokenized_tweets]
encoded_tweets

maxNum = len(vocab) + 1
binary_encoded_tweets = [[1 if (n in t) else 0 for n in range(maxNum)] for t in encoded_tweets]